In [1]:
import pandas as pd
import pymer4.models
import numpy as np
import scipy.stats

In [2]:
measures = pd.read_csv("measures.csv")
measures["d_condition_z"] = (measures["d_condition"]-measures["d_condition"].mean())/measures["d_condition"].std()
measures["tta_condition_z"] = (measures["tta_condition"]-measures["tta_condition"].mean())/measures["tta_condition"].std()
measures["tta_or_condition_z"] = (measures["tta_or_condition"]-measures["tta_or_condition"].mean())/measures["tta_or_condition"].std()
measures["dwell_time_z"] = (measures["dwell_time"]-measures["dwell_time"].mean())/measures["dwell_time"].std()

In [74]:
column_names = {"Estimate": "$\\beta$", "Z-stat": "$z$", "P-val": "$p$", "T-stat": "$t$", "F-stat": "$F$", "2.5_ci": "CI 2.5\%", "97.5_ci": "CI 97.5\%", "NumDF": "df"}
var_names = {"tta_condition_z": "$\\textrm{TTA}$",
             "d_condition_z": "distance",
             "tta_or_condition_z": "time budget",
             "dwell_time_z": "\% dwell time mirror",
             "tta_condition_z:tta_or_condition_z": "$\\textrm{TTA}$:time budget",
             "tta_condition_z:dwell_time_z": "$\\textrm{TTA}$:\% dwell time mirror",
             "tta_or_condition_z:dwell_time_z": "time budget:\% dwell time mirror",
             "tta_condition_z:tta_or_condition_z:dwell_time_z": "$\\textrm{TTA}$:time budget:\% dwell time mirror",
             "decision:d_condition_z": "decision:distance",
             "decision:tta_condition_z": "decision:$\\textrm{TTA}$",
             "decision:tta_or_condition_z": "decision:time budget",
             "decision:tta_condition_z:tta_or_condition_z": "decision:$\\textrm{TTA}$:time budget"}

def p_formatted(p):
    if p>0.01:
        return "{:.2f}".format(p)
    elif p>0.001:
        return "{:.3f}".format(p)
    else:
        return "$<0.001$"

# Dwell time

In [7]:
model_dwell_time = pymer4.models.Lmer("dwell_time ~ RT*decision + (1 | subj_id) ", data=measures, family="gaussian")
model_dwell_time_fit = model_dwell_time.fit(summarize=True)
model_dwell_time.coefs

Linear mixed model fit by REML [’lmerMod’]
Formula: dwell_time~RT*decision+(1|subj_id)

Family: gaussian	 Inference: parametric

Number of observations: 8634	 Groups: {'subj_id': 24.0}

Log-likelihood: 5549.427 	 AIC: -11086.854

Random effects:

                 Name    Var    Std
subj_id   (Intercept)  0.017  0.131
Residual               0.016  0.126

No random effect correlations specified

Fixed effects:


,Estimate,2.5_ci,97.5_ci,SE,DF,T-stat,P-val,Sig
(Intercept),0.754781,0.701355,0.808206,0.027258,24.788643,27.689943,3.657287e-20,***
RT,-0.057524,-0.064805,-0.050243,0.003715,8625.113433,-15.485115,2.276965e-53,***
decisionWait,-0.009403,-0.027541,0.008735,0.009254,8612.478371,-1.016047,3.096357e-01,
RT:decisionWait,0.015998,0.006085,0.025910,0.005057,8612.813952,3.163214,1.565800e-03,**


# Decision outcome

In [75]:
model_decision = pymer4.models.Lmer("is_gap_accepted ~ d_condition_z + tta_condition_z + tta_or_condition_z + (tta_condition_z + tta_or_condition_z + d_condition_z | subj_id) ", data=measures, family="binomial")
model_decision_fit = model_decision.fit(summarize=True)
model_decision.coefs

Linear mixed model fit by maximum likelihood  ['lmerMod']
Formula: is_gap_accepted~d_condition_z+tta_condition_z+tta_or_condition_z+(tta_condition_z+tta_or_condition_z+d_condition_z|subj_id)

Family: binomial	 Inference: parametric

Number of observations: 8634	 Groups: {'subj_id': 24.0}

Log-likelihood: -3232.833 	 AIC: 6493.667

Random effects:

                       Name    Var    Std
subj_id         (Intercept)  1.621  1.273
subj_id     tta_condition_z  0.268  0.518
subj_id  tta_or_condition_z  0.202  0.449
subj_id       d_condition_z  0.251  0.501

                        IV1                 IV2   Corr
subj_id         (Intercept)     tta_condition_z -0.124
subj_id         (Intercept)  tta_or_condition_z -0.220
subj_id         (Intercept)       d_condition_z  0.215
subj_id     tta_condition_z  tta_or_condition_z -0.140
subj_id     tta_condition_z       d_condition_z -0.548
subj_id  tta_or_condition_z       d_condition_z  0.462

Fixed effects:


,Estimate,2.5_ci,97.5_ci,SE,OR,OR_2.5_ci,OR_97.5_ci,Prob,Prob_2.5_ci,Prob_97.5_ci,Z-stat,P-val,Sig
(Intercept),1.299452,0.780160,1.818745,0.264950,3.667288,2.181821,6.164116,0.785743,0.685715,0.860415,4.904520,9.365625e-07,***
d_condition_z,0.537341,0.325559,0.749123,0.108054,1.711451,1.384805,2.115145,0.631194,0.580678,0.678988,4.972894,6.596063e-07,***
tta_condition_z,1.841681,1.611953,2.071408,0.117210,6.307130,5.012592,7.935991,0.863147,0.833682,0.888093,15.712650,1.238910e-55,***
tta_or_condition_z,-0.416692,-0.608406,-0.224978,0.097815,0.659224,0.544218,0.798534,0.397309,0.352423,0.443992,-4.259998,2.044290e-05,***


In [76]:
df = model_decision.coefs.loc[:, ["Estimate", "SE", "Z-stat", "P-val"]]

df["P-val"] = df["P-val"].apply(p_formatted)

styler = df.rename(columns=column_names, index=var_names).style.format(precision=2)

with open('C:\\Users\\azgonnikov\\Dropbox\\Apps\\Overleaf\\AAP Dynamics of merging decisions Elsevier template\\tab_decision.tex', 'w') as tf:
     tf.write(styler.to_latex(
         column_format="rrrrr", position="h", position_float="centering",
         hrules=True, label="tab:decision", caption="Coefficients of the mixed-effects logistic regression describing the final decision as a function of $\\textrm{TTA}$ and distance to the overtaking vehicle and the time budget provided by the merging lane. All effects were modelled as random slopes per participant."
     )
)

# Response time

In [3]:
model_RT = pymer4.models.Lmer("RT ~ 1 + decision*(d_condition_z + tta_condition_z*tta_or_condition_z) + (decision + tta_condition_z +  tta_or_condition_z| subj_id) ", data=measures, family="gaussian")
model_RT.fit(summarize=True, factors={"decision": ["Merge", "Wait"]})
model_RT.coefs
RT_anova = model_RT.anova()
RT_decision_marginal_estimates, RT_decision_comparisons = model_RT.post_hoc(marginal_vars=["decision"])

Linear mixed model fit by REML [’lmerMod’]
Formula: RT~1+decision*(d_condition_z+tta_condition_z*tta_or_condition_z)+(decision+tta_condition_z+tta_or_condition_z|subj_id)

Family: gaussian	 Inference: parametric

Number of observations: 8634	 Groups: {'subj_id': 24.0}

Log-likelihood: -5153.224 	 AIC: 10348.448

Random effects:

                        Name    Var    Std
subj_id          (Intercept)  0.144  0.379
subj_id         decisionWait  0.083  0.287
subj_id      tta_condition_z  0.002  0.048
subj_id   tta_or_condition_z  0.007  0.082
Residual                      0.186  0.431

                     IV1                 IV2   Corr
subj_id      (Intercept)        decisionWait -0.494
subj_id      (Intercept)     tta_condition_z  0.282
subj_id      (Intercept)  tta_or_condition_z  0.784
subj_id     decisionWait     tta_condition_z -0.150
subj_id     decisionWait  tta_or_condition_z -0.416
subj_id  tta_condition_z  tta_or_condition_z  0.274

Fixed effects:

SS Type III Analysis of Varia

In [4]:
RT_anova

,SS,MS,NumDF,DenomDF,F-stat,P-val,Sig
decision,21.950024,21.950024,1,23.295020,118.239584,1.325535e-10,***
d_condition_z,0.253921,0.253921,1,8565.325964,1.367813,2.422205e-01,
tta_condition_z,13.534993,13.534993,1,26.175459,72.909801,4.829125e-09,***
tta_or_condition_z,3.720726,3.720726,1,26.422380,20.042672,1.295146e-04,***
tta_condition_z:tta_or_condition_z,7.082923,7.082923,1,8087.385956,38.154031,6.853331e-10,***
decision:d_condition_z,5.999752,5.999752,1,8565.474244,32.319245,1.351015e-08,***
decision:tta_condition_z,14.530222,14.530222,1,1107.769860,78.270867,3.481564e-18,***
decision:tta_or_condition_z,6.220185,6.220185,1,6805.134404,33.506665,7.418040e-09,***
decision:tta_condition_z:tta_or_condition_z,0.977408,0.977408,1,7488.086760,5.265063,2.178517e-02,*


In [107]:
df = RT_anova.loc[:, ["SS", "MS", "NumDF", "F-stat", "P-val"]]
df["P-val"] = df["P-val"].apply(p_formatted)
styler = df.rename(columns=column_names, index=var_names).style.format(precision=2)

with open('C:\\Users\\azgonnikov\\Dropbox\\Apps\\Overleaf\\AAP Dynamics of merging decisions Elsevier template\\tab_RT.tex', 'w') as tf:
     tf.write(styler.to_latex(
         column_format="rrrrrrr", position="h", position_float="centering",
         hrules=True, label="tab:RT", caption="ANOVA table based on the mixed-effects linear regression describing response time as a function of $\\textrm{TTA}$ and distance to the overtaking vehicle and the time budget provided by the merging lane. Random slopes of $\\textrm{TTA}$ and distance to the overtaking vehicle and the merging lane time budget were included per participant."
     )
)

In [99]:
# RT_decision_marginal_estimates, RT_decision_comparisons = model_RT.post_hoc(marginal_vars=["tta_condition_z"], grouping_vars=["decision"])
RT_decision_marginal_estimates, RT_decision_comparisons = model_RT.post_hoc(marginal_vars=["decision"])
RT_decision_comparisons

,Contrast,Estimate,2.5_ci,97.5_ci,SE,DF,T-stat,P-val,Sig
1,Merge - Wait,-0.66,-0.786,-0.535,0.061,23.295,-10.875,0.0,***


In [100]:
def get_marginal_estimates(model_RT, marginal_vars):
    marginal_estimates, comparisons = model_RT.post_hoc(marginal_vars=marginal_vars, grouping_vars=["decision"])
    marginal_estimates["T-stat"] = marginal_estimates["Estimate"]/marginal_estimates["SE"]
    marginal_estimates["P-val"] = scipy.stats.t.sf(np.abs(marginal_estimates["T-stat"]), marginal_estimates.DF)
    return marginal_estimates

In [101]:
get_marginal_estimates(model_RT, "d_condition_z")

,decision,Estimate,2.5_ci,97.5_ci,SE,DF,T-stat,P-val
1,Merge,-0.023,-0.034,-0.011,0.006,8553.857,-3.833333,0.000064
2,Wait,0.035,0.018,0.051,0.008,8569.662,4.375000,0.000006


In [102]:
get_marginal_estimates(model_RT, "tta_condition_z")

,decision,Estimate,2.5_ci,97.5_ci,SE,DF,T-stat,P-val
1,Merge,0.035,0.010,0.060,0.012,24.588,2.916667,3.722613e-03
2,Wait,0.174,0.141,0.207,0.017,74.541,10.235294,3.705420e-16


In [103]:
get_marginal_estimates(model_RT, "tta_or_condition_z")

,decision,Estimate,2.5_ci,97.5_ci,SE,DF,T-stat,P-val
1,Merge,0.040,0.003,0.077,0.018,25.673,2.222222,1.764791e-02
2,Wait,0.123,0.081,0.165,0.021,45.894,5.857143,2.387071e-07


In [104]:
marginal_estimates, comparisons = model_RT.post_hoc(marginal_vars=["tta_or_condition_z"], grouping_vars=["decision", "tta_condition_z"])
marginal_estimates["T-stat"] = marginal_estimates["Estimate"]/marginal_estimates["SE"]
marginal_estimates["P-val"] = scipy.stats.t.sf(np.abs(marginal_estimates["T-stat"]), marginal_estimates.DF)

In [105]:
marginal_estimates

,decision,tta_condition_z,Estimate,2.5_ci,97.5_ci,SE,DF,T-stat,P-val
1,Merge,-1.0,0.014,-0.027,0.054,0.020,39.276,0.700000,2.440265e-01
2,Wait,-1.0,0.065,0.026,0.104,0.019,31.981,3.421053,8.618330e-04
3,Merge,1.0,0.066,0.029,0.104,0.018,26.568,3.666667,5.405259e-04
4,Wait,1.0,0.181,0.125,0.236,0.028,151.822,6.464286,6.565168e-10


# Including dwell time in the model for decision outcome

In [78]:
model_decision_dwell = pymer4.models.Lmer("is_gap_accepted ~ d_condition_z + tta_condition_z*tta_or_condition_z*dwell_time_z + (tta_condition_z + d_condition_z + tta_or_condition_z | subj_id) ", data=measures, family="binomial")
model_decision_dwell_fit = model_decision_dwell.fit(summarize=True)
model_decision_dwell.coefs

Linear mixed model fit by maximum likelihood  ['lmerMod']
Formula: is_gap_accepted~d_condition_z+tta_condition_z*tta_or_condition_z*dwell_time_z+(tta_condition_z+d_condition_z+tta_or_condition_z|subj_id)

Family: binomial	 Inference: parametric

Number of observations: 8634	 Groups: {'subj_id': 24.0}

Log-likelihood: -3213.004 	 AIC: 6464.008

Random effects:

                       Name    Var    Std
subj_id         (Intercept)  1.657  1.287
subj_id     tta_condition_z  0.261  0.511
subj_id       d_condition_z  0.253  0.503
subj_id  tta_or_condition_z  0.114  0.338

                     IV1                 IV2   Corr
subj_id      (Intercept)     tta_condition_z -0.132
subj_id      (Intercept)       d_condition_z  0.219
subj_id      (Intercept)  tta_or_condition_z -0.138
subj_id  tta_condition_z       d_condition_z -0.540
subj_id  tta_condition_z  tta_or_condition_z -0.206
subj_id    d_condition_z  tta_or_condition_z  0.580

Fixed effects:


,Estimate,2.5_ci,97.5_ci,SE,OR,OR_2.5_ci,OR_97.5_ci,Prob,Prob_2.5_ci,Prob_97.5_ci,Z-stat,P-val,Sig
(Intercept),1.277144,0.751927,1.802361,0.267973,3.586382,2.121083,6.063945,0.781963,0.679598,0.858436,4.765948,1.879680e-06,***
d_condition_z,0.542184,0.329556,0.754812,0.108486,1.719759,1.390351,2.127212,0.632320,0.581651,0.680226,4.997754,5.800192e-07,***
tta_condition_z,1.828735,1.601117,2.056353,0.116134,6.226004,4.958567,7.817406,0.861611,0.832174,0.886588,15.746795,7.225156e-56,***
tta_or_condition_z,-0.391190,-0.546368,-0.236013,0.079174,0.676251,0.579049,0.789770,0.403431,0.366707,0.441269,-4.940920,7.775490e-07,***
dwell_time_z,0.062410,-0.037146,0.161967,0.050795,1.064399,0.963535,1.175821,0.515597,0.490714,0.540403,1.228663,2.191980e-01,
tta_condition_z:tta_or_condition_z,0.049566,-0.032159,0.131291,0.041697,1.050815,0.968353,1.140299,0.512389,0.491961,0.532776,1.188719,2.345503e-01,
tta_condition_z:dwell_time_z,0.066278,-0.029596,0.162151,0.048916,1.068523,0.970838,1.176038,0.516563,0.492602,0.540449,1.354929,1.754400e-01,
tta_or_condition_z:dwell_time_z,0.239457,0.150385,0.328528,0.045445,1.270559,1.162282,1.388922,0.559580,0.537526,0.581401,5.269113,1.370847e-07,***
tta_condition_z:tta_or_condition_z:dwell_time_z,-0.046320,-0.121023,0.028382,0.038114,0.954736,0.886014,1.028789,0.488422,0.469781,0.507095,-1.215300,2.242518e-01,


In [79]:
df = model_decision_dwell.coefs.loc[:, ["Estimate", "SE", "Z-stat", "P-val"]]

df["P-val"] = df["P-val"].apply(p_formatted)

styler = df.rename(columns=column_names, index=var_names).style.format(precision=2)

with open('C:\\Users\\azgonnikov\\Dropbox\\Apps\\Overleaf\\AAP Dynamics of merging decisions Elsevier template\\tab_decision_dwell.tex', 'w') as tf:
     tf.write(styler.to_latex(
         column_format="rrrrr", position="h", position_float="centering",
         hrules=True, label="tab:decision_dwell", caption="Coefficients of the mixed-effects logistic regression describing the final decision as a function of dwell time. Random slopes of $\\textrm{TTA}$ and distance to the overtaking vehicle and the time budget provided by the merging lane were included per participant."
     )
)